<a href="https://colab.research.google.com/github/kimimumemo/nanem/blob/main/Nanem_SupportVectorMachine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/client/session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
#Convolutional Neural Network

#Importing the libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.4.1'

In [ ]:
#Part 1 - Data Preprocessing

#Preprocessing the Training Set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/gambar_tanah (1)/train',
                                                 target_size = (64,64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

#Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/gambar_tanah (1)/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 228 images belonging to 2 classes.
Found 5 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense

In [ ]:
from tensorflow.keras.regularizers import l2

In [ ]:
#Part 2 - Building the CNN
#Initialising the CNN
cnn = tf.keras.models.Sequential()

#Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, 
                               padding="same",
                               kernel_size=3, 
                               activation='relu', 
                               strides=2, 
                               input_shape=[64,64, 3]))

#Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,
                                  strides=2))

#Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32,
                               padding='same',
                               kernel_size=3,
                               activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,
                                  strides=2))

#Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

#Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))


In [ ]:
##for multiclassification
cnn.add(Dense(2, kernel_regularizer=tf.keras.regularizers.l2(0.01), activation='softmax'))
cnn.compile(optimizer='adam', loss='squared_hinge', metrics=['accuracy'])

In [ ]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 5

In [ ]:
#Part 3 -Training the CNN

#Compiling the CNN
cnn.compile(optimizer='adam', loss='hinge', metrics=['accuracy'])

#Training the CNN on the Training set and evaluating it on the Test set
r=cnn.fit(x=training_set, validation_data=test_set,
          epochs=10)

Epoch 1/10
8/8 [==============================] - 4s 518ms/step - loss: 1.1275 - accuracy: 0.3094 - val_loss: 1.0677 - val_accuracy: 0.6000
Epoch 2/10
8/8 [==============================] - 3s 409ms/step - loss: 1.1186 - accuracy: 0.3086 - val_loss: 1.0638 - val_accuracy: 0.6000
Epoch 3/10
8/8 [==============================] - 3s 424ms/step - loss: 1.1092 - accuracy: 0.3135 - val_loss: 1.0605 - val_accuracy: 0.6000
Epoch 4/10
8/8 [==============================] - 3s 436ms/step - loss: 1.1021 - accuracy: 0.3101 - val_loss: 1.0579 - val_accuracy: 0.6000
Epoch 5/10
8/8 [==============================] - 3s 414ms/step - loss: 1.0972 - accuracy: 0.2917 - val_loss: 1.0558 - val_accuracy: 0.6000
Epoch 6/10
8/8 [==============================] - 3s 406ms/step - loss: 1.0889 - accuracy: 0.2995 - val_loss: 1.0539 - val_accuracy: 0.6000
Epoch 7/10
8/8 [==============================] - 3s 475ms/step - loss: 1.0817 - accuracy: 0.3045 - val_loss: 1.0525 - val_accuracy: 0.6000
Epoch 8/10
8/8 [====

In [ ]:
#plot the loss
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history)

In [ ]:
import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
 
def create_graph(model_path):
    """
    create_graph loads the inception model to memory, should be called before
    calling extract_features.
 
    model_path: path to inception model in protobuf form.
    """
    with gfile.FastGFile(model_path, 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')
 
 
def extract_features(image_paths, verbose=False):
    """
    extract_features computed the inception bottleneck feature for a list of images
 
    image_paths: array of image path
    return: 2-d array in the shape of (len(image_paths), 2048)
    """
    feature_dimension = 2048
    features = np.empty((len(image_paths), feature_dimension))
 
    with tf.compat.v1.Session() as sess:
        flattened_tensor = sess.graph.get_tensor_by_name('pool_3:0')
 
        for i, image_path in enumerate(image_paths):
            if verbose:
                print('Processing %s...' % (image_path))
 
            if not gfile.Exists(image_path):
                tf.logging.fatal('File does not exist %s', image)
 
            image_data = gfile.FastGFile(image_path, 'rb').read()
            feature = sess.run(flattened_tensor, {
                'DecodeJpeg/contents:0': image_data
            })
            features = np.squeeze(feature)
 
    return features 

In [ ]:
extract_features('/content/drive/MyDrive/gambar_tanah')

KeyError: ignored